In [ ]:
# Instalaciones

import requests
import pandas as pd
import random
import numpy as np
import time
from google.colab import files
%pip install bs4
from bs4 import BeautifulSoup
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
options = webdriver.ChromeOptions() # Usamos chrome, se podria usar otro.
options.add_argument('--headless') # Chromium sin interfaz grafica
options.add_argument('--no-sandbox') # Seguridad
options.add_argument('--disable-dev-shm-usage') # configuracion de linux
options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from lxml import etree
!pip install tika
from tika import parser
from google.colab import files
import time
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
nuestros_headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
    }

# Precio venta dolar santander + $10
url = 'https://www.google.com/search?q=dolar+santander+hoy+venta+argentina&rlz=1C1CHBF_esAR973AR973&oq=d&aqs=chrome.1.69i57j35i39j69i60l6.6018j0j4&sourceid=chrome&ie=UTF-8'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
dolar_hoy = soup.find_all('td')
dolar_hoy = [dolar.text for dolar in dolar_hoy]
dolar_hoy = float([dolar for dolar in dolar_hoy if '$' in dolar][-2].split('$')[-2].replace(' ','').replace(',','.')) + 10


# df Lista Trust
lista_dfs = []
for i in range(7):
  try:
    df = pd.read_excel('trust.xlsx',sheet_name=i)
    lista_dfs.append(df)
  except:
    continue
df = pd.concat(lista_dfs)
df.rename(columns = {'NUMERO DE PEDIDO :':'Producto', 'Unnamed: 3':'unidadesPorCaja', 'Unnamed: 4':'precioUnidadDolares',
                'Unnamed: 5':'precioCajaDolares'}, inplace = True)
columnas = ['Producto','unidadesPorCaja','precioUnidadDolares','precioCajaDolares']
df['Producto'] = df['Producto'].astype(str)
df = df[df['Producto'].notna()]
df = df[df['Producto'] != 'nan']
df.drop(df.index[0:8], inplace=True)
df.drop(df.index[-3:], inplace=True)
df = df[pd.to_numeric(df['precioUnidadDolares'], errors='coerce').notnull()]
df['precioUnidadDolares'] = df['precioUnidadDolares'].astype(float)
df = df[pd.to_numeric(df['unidadesPorCaja'], errors='coerce').notnull()]
df['unidadesPorCaja'] = df['unidadesPorCaja'].astype(int)
df = df[pd.to_numeric(df['precioCajaDolares'], errors='coerce').notnull()]
df['precioCajaDolares'] = round(df['precioCajaDolares'].astype(float))
df.reset_index(inplace=True)
df = df[columnas]
df['precioCajaPesos'] = round(df['precioCajaDolares']*dolar_hoy).astype(int)
df['precioUnidadPesos'] = round(df['precioUnidadDolares']*dolar_hoy).astype(int)

# Precio dolar
Producto = list(df['Producto'])
unidadesPorCaja = list(df['unidadesPorCaja'])
precioUnidadDolares = list(df['precioUnidadDolares'])
precioCajaDolares = list(df['precioCajaDolares'])
precioCajaPesos = list(df['precioCajaPesos'])
precioUnidadPesos = list(df['precioUnidadPesos'])


# Productos Buscables
productos = list(df['Producto'])

# Corregir nombres de articulos y obtencion de la version buscable
new_productos = []
productos_buscables = []
for producto in productos:
  producto = producto.replace('  ','')
  producto = producto.replace('  ','')
  producto = producto.replace(' ','-')
  producto = producto.replace('--','')
  producto = producto.replace('%','%25')
  productos_buscables.append(producto)
productos = productos_buscables.copy()



# Obtencion del precio en Mercado libre
lista_precios = []
url = 'https://listado.mercadolibre.com.ar/'
noBuscables = []
contador = 0
for i,prod in enumerate(productos):
  r = requests.get(url+prod) 
  if r.status_code == 200:
    contador += 1
    soup = BeautifulSoup(r.content, 'html.parser')
    # Precios
    dom = etree.HTML(str(soup))
    precios = dom.xpath("//li[@class='ui-search-layout__item']//div[@class='ui-search-price__second-line']//span[@class='price-tag-fraction']")
    precios = [int(i.text.replace('.','')) for i in precios][:4]
    lista_precios.append(int(np.mean(precios)))
  else:
    print('\n'*10)
    print(f'NO PUDO ACCEDERSE A PAG PRODUCTO {productos[i]}')
    print('\n'*10)
    noBuscables.append(productos[i])
    del Producto[i]
    del unidadesPorCaja[i]
    del precioUnidadDolares[i]
    del precioCajaDolares[i]
    del precioCajaPesos[i]
    del precioUnidadPesos[i]
  print(f"Buscando articulo nro {contador} de {len(productos)}")  

# Sustituto de descripciones (productos sin codigo en descripcion)
lista = [0]*(len(productos) - len(lista_precios))
lista_precios.extend(lista)

# Data Frame
df = pd.DataFrame(np.array([Producto, lista_precios,unidadesPorCaja, precioUnidadPesos, precioCajaPesos,
                            precioUnidadDolares, precioCajaDolares]).T, columns=['Producto', 'precioEstimadoVenta','unidadesPorCaja','precioUnidadPesos','precioCajaPesos',
                                                                                 'precioUnidadDolares','precioCajaDolares'])
df.precioUnidadPesos = df['precioUnidadPesos'].astype(float)
df.precioEstimadoVenta = df['precioEstimadoVenta'].astype(int)
df['Ganancia'] = round(df['precioEstimadoVenta'] - df['precioUnidadPesos']).astype(int)
df = df.sort_values('Ganancia', ascending=False)

# Reubicar columnas
Producto = df.pop('Producto')
Ganancia = df.pop('Ganancia')

df.insert(0, 'Producto', Producto)
df.insert(1, 'Ganancia', Ganancia)

df.head(60)
#df.iloc[50:60,:]

     |████████████████████████████████| 979 kB 4.2 MB/s 
     |████████████████████████████████| 359 kB 35.2 MB/s 
     |████████████████████████████████| 138 kB 40.7 MB/s 
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 4.0 MB 36.2 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,734 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archi

In [ ]:
# <----- Apretar Play para descargarlo como excel
df.to_csv('Ganancias_Trust.csv')
files.download("Ganancias_Trust.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>